In [36]:
with open("shake.txt", "r") as f:
    shake = f.read()

shake = shake[:100_000]
enc = list(shake.encode("utf-8"))

shake[:20]

'First Citizen:\nBefor'

In [37]:
print(enc[:20])

[70, 105, 114, 115, 116, 32, 67, 105, 116, 105, 122, 101, 110, 58, 10, 66, 101, 102, 111, 114]


In [39]:
from collections import OrderedDict


with open("shake.txt", "r") as f:
    shake = f.read()

shake = shake[:100_000]

enc = list(shake.encode("utf-8"))


def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]):  # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts


def get_most_common_pair(enc):
    stats = get_stats(enc)
    count, pair = max([(c, p) for p, c in get_stats(enc).items()])
    return count, pair


def replace_pair_with_key(seq, mcp, key):
    i = 0
    while i < len(seq) - 1:
        if seq[i] == mcp[0] and seq[i + 1] == mcp[1]:
            seq[i] = key
            del seq[i + 1]
            i += 1
        i += 1
    return seq


def replace_key_with_pair(seq, key, pair):
    i = 0
    while i < len(seq):
        if seq[i] == key:
            seq[i : i + 1] = pair
            i += len(pair) - 1
        i += 1
    return seq


target_vocab_size = 260
current_vocab_size = 256
starting_len = len(enc)
decoding_dict = []

print(f"{len(enc):_} lenght")
print(f"{current_vocab_size:_} vocab_size")
print("-" * 30)

count, mcp = get_most_common_pair(enc)

for i in range(target_vocab_size - current_vocab_size):
    decoding_dict.append((current_vocab_size, mcp))
    enc = replace_pair_with_key(enc, mcp, current_vocab_size)
    count, mcp = get_most_common_pair(enc)
    current_vocab_size += 1


print(f"{len(enc):_} lenght")
print(f"{current_vocab_size:_} vocab_size")
print(f"{starting_len/len(enc):.02f}x compression ratio")
print("-" * 30)

100_000 lenght
256 vocab_size
------------------------------
92_388 lenght
260 vocab_size
1.08x compression ratio
------------------------------


In [41]:
print(*decoding_dict)

(256, (101, 32)) (257, (116, 104)) (258, (116, 32)) (259, (115, 32))


In [30]:
def encode(s):
    enc = list(s.encode("utf-8"))
    for key, pair in decoding_dict:
        enc = replace_pair_with_key(enc, pair, key)
    return enc


encoded = encode(shake)
print(encoded)

[70, 105, 114, 115, 258, 67, 105, 287, 122, 268, 266, 66, 101, 102, 270, 256, 119, 256, 112, 114, 111, 99, 101, 101, 262, 264, 272, 102, 117, 114, 257, 263, 261, 104, 101, 273, 32, 109, 256, 115, 112, 292, 107, 278, 65, 276, 266, 83, 112, 292, 107, 261, 115, 112, 292, 107, 278, 70, 105, 114, 115, 258, 67, 105, 287, 122, 268, 266, 89, 260, 32, 273, 256, 97, 289, 114, 281, 111, 108, 118, 101, 262, 114, 97, 257, 263, 32, 294, 100, 105, 256, 257, 264, 32, 294, 102, 97, 109, 105, 115, 104, 63, 269, 65, 276, 266, 82, 281, 111, 108, 118, 101, 100, 46, 32, 114, 281, 111, 108, 118, 101, 100, 278, 70, 105, 114, 115, 258, 67, 105, 287, 122, 268, 266, 70, 105, 114, 115, 116, 261, 277, 32, 107, 286, 119, 32, 67, 97, 105, 117, 259, 77, 273, 99, 105, 117, 259, 105, 259, 99, 283, 101, 102, 32, 268, 101, 109, 272, 116, 111, 285, 112, 101, 111, 112, 108, 101, 278, 65, 276, 266, 87, 256, 107, 286, 119, 39, 116, 261, 119, 256, 107, 286, 119, 39, 116, 278, 70, 105, 114, 115, 258, 67, 105, 287, 122, 268, 26

In [31]:
def decode(ids):
    for key, pair in decoding_dict[::-1]:
        ids = replace_key_with_pair(ids, key, pair)
    return "".join(map(chr, ids))


print(decode(encoded)[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [1]:
chr(32)

' '